# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path
import api_keys
import hvplot.pandas


In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df= pd.read_csv("output_data_cities.csv")

#Converting temperature to Celcius
city_data_df["Max Temp"]=city_data_df["Max Temp"]-273.15

#Renaming Coloumns
city_data_df=city_data_df.rename(columns={
    "Max Temp":"Max Temp(C)",
    "Humidity":"Humidity(%)",
    "Cloudiness":"Cloudiness(%)",
    "Wind Speed":"Wind Speed(mph)"    
    
})

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp(C),Humidity(%),Cloudiness(%),Wind Speed(mph),Country,Date
0,0,utrik,11.2278,169.8474,27.44,78,98,10.34,MH,1701925936
1,1,tupa,-21.9347,-50.5136,24.96,65,32,1.58,BR,1701925936
2,2,blackmans bay,-43.0167,147.3167,23.07,57,100,0.45,AU,1701925936
3,3,puerto natales,-51.7236,-72.4875,4.30,92,0,1.93,CL,1701925850
4,4,papatowai,-46.5619,169.4708,13.80,56,97,6.12,NZ,1701925937


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

humidity = city_data_df["Humidity(%)"].astype(int)
# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity(%)",      # This line sets the size based on the "Humidity" column
    color="City",
    alpha=.75
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity(%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp(C)"] < 27) & (city_data_df["Max Temp(C)"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed(mph)"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness(%)"] == 0)]

# Drop any rows with null values
# YOUR CODE HERE
city_data_df = city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
city_data_df.sample(5)

,City_ID,City,Lat,Lng,Max Temp(C),Humidity(%),Cloudiness(%),Wind Speed(mph),Country,Date
331,331,san sebastian de la gomera,28.0916,-17.1133,21.20,77,0,1.03,ES,1701926063
316,316,palmas,-10.2128,-48.3603,25.93,83,0,1.54,BR,1701925832
91,91,niafunke,15.9322,-3.9906,21.94,17,0,4.48,ML,1701925961
395,395,bonthe,7.5264,-12.5050,25.52,87,0,0.72,SL,1701926087
421,421,wajir,1.7471,40.0573,26.61,57,0,4.21,KE,1701926095


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity(%)"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity(%)"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.sample(5)

,City,Country,Lat,Lng,Humidity(%),Hotel Name
223,moyale,KE,3.5167,39.0584,42,
133,narwana,IN,29.6167,76.1167,28,
375,san jose del cabo,MX,23.0589,-109.6977,78,
480,kasane,BW,-17.8167,25.1500,61,
331,san sebastian de la gomera,ES,28.0916,-17.1133,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":api_keys.geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
niafunke - nearest hotel: No hotel found
narwana - nearest hotel: No hotel found
moyale - nearest hotel: Paul’s guest house
palmas - nearest hotel: Hotel Triângulo Mineiro
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
san jose del cabo - nearest hotel: Hotel Posada Terranova
bonthe - nearest hotel: No hotel found
terenos - nearest hotel: No hotel found
wajir - nearest hotel: Kilimanjaro
kasane - nearest hotel: Kasane Travel Lodge


,City,Country,Lat,Lng,Humidity(%),Hotel Name
91,niafunke,ML,15.9322,-3.9906,17,No hotel found
133,narwana,IN,29.6167,76.1167,28,No hotel found
223,moyale,KE,3.5167,39.0584,42,Paul’s guest house
316,palmas,BR,-10.2128,-48.3603,83,Hotel Triângulo Mineiro
331,san sebastian de la gomera,ES,28.0916,-17.1133,77,Hotel Villa Gomera
375,san jose del cabo,MX,23.0589,-109.6977,78,Hotel Posada Terranova
395,bonthe,SL,7.5264,-12.5050,87,No hotel found
409,terenos,BR,-20.4422,-54.8603,73,No hotel found
421,wajir,KE,1.7471,40.0573,57,Kilimanjaro
480,kasane,BW,-17.8167,25.1500,61,Kasane Travel Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity(%)",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity(%),Hotel Name,Country)